In [18]:
from utils import loadAllTSPs, calcDist_EUC, GA_cost, create_GA_fitness, create_clustering_fitness, clustering

df = loadAllTSPs('./tsplib/')
# df=df[2:3]

In [19]:
import pygad
import numpy as np
import os
import time
import warnings
warnings.filterwarnings("ignore")

num_generations = 100
sol_per_pop = 20

In [20]:
# check if benchmarks exists
if not os.path.exists('benchmarks'):
    os.mkdir('benchmarks')
# check if GA.csv exists inside benchmarks
if not os.path.exists('benchmarks/GA.csv'):
    with open('benchmarks/GA.csv', 'w') as f:
        f.write('i, cost, time, fitness, no_of_hubs\n')
# check if clustering.csv exists inside benchmarks
if not os.path.exists('benchmarks/clustering.csv'):
    with open('benchmarks/clustering.csv', 'w') as f:
        f.write('i, cost, time, fitness, no_of_hubs\n')

In [21]:
for i, data in enumerate(df):
    try:
        # compute distance matrix
        distMat = calcDist_EUC(data)
        # do NN with GA using pygad
        t1 = time.time()
        ga_instance = pygad.GA(gene_space=[0, 1],
                        num_generations=num_generations,
                        fitness_func=create_GA_fitness(data, distMat, tsp_heuristic="NN"),
                        num_parents_mating=20,
                        num_genes=len(data),
                        sol_per_pop=sol_per_pop,
                        crossover_type="single_point",
                        mutation_type="random",
                        random_seed=42,
                        on_generation=None,
                        )
        ga_instance.run()
        # get the best solution after iterating finishing all generations.
        solution, solution_fitness, solution_idx = ga_instance.best_solution()
        cost = GA_cost(data, solution, distMat)
        t2 = time.time()
        hubs = np.sum(solution)
        # Make a csv string of i, cost, time, fitness, number of hubs
        csv_str = f"{i},{cost},{t2-t1},{solution_fitness},{hubs}\n"
        # write the csv string to the file
        with open('benchmarks/GA.csv', 'a') as f:
            f.write(csv_str)
        # do clustering with GA
        t1 = time.time()
        ga_instance = pygad.GA(
            num_generations=num_generations,
            num_parents_mating=10,
            fitness_func=create_clustering_fitness(data, distMat, "NN"),
            sol_per_pop=sol_per_pop,
            num_genes=1,
            init_range_low=3,
            init_range_high=len(data)*3//4,
            gene_type=int,
            parent_selection_type="sss",
            keep_parents=2,
            crossover_type="single_point",
            mutation_type=None,
            random_seed=42,
            on_generation=None
        )
        ga_instance.run()
        # get the best solution after iterating finishing all generations.
        solution, solution_fitness, solution_idx = ga_instance.best_solution()
        (clusters, ordered_hubs, total_cost)=clustering(data, distMat, solution[0], "NN", True)
        t2 = time.time()
        hubs = solution[0]
        # Make a csv string of i, cost, time, fitness, number of hubs
        csv_str = f"{i},{total_cost},{t2-t1},{solution_fitness},{hubs}\n"
        # write the csv string to the file
        with open('benchmarks/clustering.csv', 'a') as f:
            f.write(csv_str)
    except Exception as e:
        print(f"Error in index {i}: {str(e)}")
        with open('./benchmarks/error_log.txt', 'a') as f:
            f.write(f"Error in index {i}: {str(e)}\n")

attempt to get argmin of an empty sequence
Traceback (most recent call last):
  File "c:\Github\Ring-Star-Problem-Study\.venv\Lib\site-packages\pygad\pygad.py", line 1708, in cal_pop_fitness
    fitness = self.fitness_func(self, sol, sol_idx)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Github\Ring-Star-Problem-Study\utils.py", line 301, in GA_fitness
    return 100/GA_cost(data, solution, dist_matrix, HUB_EDGE_COST, NON_HUB_EDGE_COST, tsp_heuristic)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Github\Ring-Star-Problem-Study\utils.py", line 254, in GA_cost
    assignments = nodeAlloc_NN(
                  ^^^^^^^^^^^^^
  File "c:\Github\Ring-Star-Problem-Study\utils.py", line 230, in nodeAlloc_NN
    nearest_hub_index = hub_indices[np.argmin(distances_to_hubs)]
                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Github\Ring-Star-Problem-Study\.venv\Lib\site-packages\numpy\core\f

Error in index 8: attempt to get argmin of an empty sequence
